In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mlp
import random
from pathlib import Path
import os
from operator import itemgetter

import tensorflow.experimental.numpy as tnp
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from numpy import savez_compressed
from numpy import load 
from numpy.random import randn


import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Reshape
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.applications import VGG16


# Load .npz files

In [2]:
# #### combining the npz
npz_path = "E:/UCF_Crimes/npz_120/"
npz_list = os.listdir(npz_path)

In [3]:
npz_list

['crimes_120_0.npz',
 'crimes_120_1.npz',
 'crimes_120_10.npz',
 'crimes_120_11.npz',
 'crimes_120_12.npz',
 'crimes_120_2.npz',
 'crimes_120_3.npz',
 'crimes_120_4.npz',
 'crimes_120_5.npz',
 'crimes_120_6.npz',
 'crimes_120_7.npz',
 'crimes_120_8.npz',
 'crimes_120_9.npz']

In [4]:
full_npz = itemgetter(0,1,3,4)(npz_list)

In [5]:
full_npz = [npz_path+npz for npz in full_npz]
full_npz

['E:/UCF_Crimes/npz_120/crimes_120_0.npz',
 'E:/UCF_Crimes/npz_120/crimes_120_1.npz',
 'E:/UCF_Crimes/npz_120/crimes_120_11.npz',
 'E:/UCF_Crimes/npz_120/crimes_120_12.npz']

In [6]:
data_all = [np.load(fname) for fname in full_npz] 

In [7]:
images = list()
images += [npz['arr_0'] for npz in data_all]

In [8]:
labels = list()
labels += [npz['arr_1'] for npz in data_all]

# Before all, define a custom activation function

In [9]:
# custom activation function
def custom_activation(output):
    logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
    result = logexpsum / (logexpsum + 1.0)
    return result

# Defining the models.

## Discriminator

In [10]:
# define the standalone supervised and unsupervised discriminator models
def define_discriminator(in_shape=(120, 120, 3), n_classes=14):
    init = RandomNormal(stddev=0.02)
# image input
    in_image = Input(shape=in_shape)
#downsample
    fe = Conv2D(32, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    fe = BatchNormalization(axis=1)(fe)
    fe = LeakyReLU(alpha=0.4)(fe)
# # downsample
    fe = Conv2D(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization(axis=1)(fe)
    fe = LeakyReLU(alpha=0.4)(fe)
# #downsample
    fe = Conv2D(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization(axis=1)(fe)
    fe = LeakyReLU(alpha=0.4)(fe)
# downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization(axis=1)(fe)
    fe = LeakyReLU(alpha=0.4)(fe)
    
# flatten feature maps
    fe = Flatten()(fe)
# dropout
    fe = Dropout(0.5)(fe)
# output layer nodes
    fe = Dense(n_classes)(fe)#kernel_initializer=init)(fe)
# supervised output
    c_out_layer = Activation('softmax')(fe)
# define and compile supervised discriminator model
    c_model = Model(in_image, c_out_layer)
    c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.006, beta_1=0.5), metrics=['accuracy'])
# unsupervised output
    d_out_layer = Lambda(custom_activation)(fe)
# define and compile unsupervised discriminator model
    d_model = Model(in_image, d_out_layer)
    d_model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return d_model, c_model

##  Generator

In [11]:
# define the standalone generator model
def define_generator(latent_dim):
    init = RandomNormal(stddev=0.02)
    #image generator input
    in_lat = Input(shape=(latent_dim,))
# foundation for 120x120 image (the 15*15 is the 1/8 of the size of the image. metric given by the book)
    n_nodes = 128 * 15 * 15
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((15, 15, 128))(gen)
    # upsample to 30x30
    gen = Conv2DTranspose(32, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    gen = BatchNormalization(axis=1)(gen)
    gen = LeakyReLU(alpha=0.4)(gen)
#upsample to 60x60
    gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    gen = BatchNormalization(axis=1)(gen)
    gen = LeakyReLU(alpha=0.4)(gen)
# #upsample to 120x120
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    gen = BatchNormalization(axis=1)(gen)
    gen = LeakyReLU(alpha=0.4)(gen)
# output
    out_layer = Conv2D(3, (15, 15), activation='tanh', padding='same', kernel_initializer=init)(gen)
# define model 
    model = Model(in_lat, out_layer)
    return model

### combined generator and discriminator model (for updating the generator) 

In [12]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
# make weights in the discriminator not trainable
    d_model.trainable = False
# connect image output from generator as input to discriminator
    gan_output = d_model(g_model.output)
# define gan model as taking noise and outputting a classification
    model = Model(g_model.input, gan_output)
# compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='mse', optimizer=opt)
    return model

# select a supervised subset of the dataset

In [13]:
def select_supervised_samples(dataset, n_samples=1200, n_classes=14):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for n in range (0, 4):
        for i in range(n_classes):
            # get all images for this class
            X_with_class = [X[n]][[y[n]] == i]
            # choose random instances
            ix = randint(0, len(X_with_class), n_per_class)
            # add to list
            [X_list.append(X_with_class[j]) for j in ix]
            [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list) 

# Select random samples from the supervised dataset

In [14]:
def generate_real_samples(X, y, n_samples):
# split into images and labels
    images = X
    labels = y
# choose random instances
    ix = randint(0, images.shape[0], n_samples)
# select images and labels
    X, labels = images[ix], labels[ix]
# generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

In [15]:
def generate_real_samples2(x, y, n_samples):
# split into images and labels
    ix = randint(0, x[0].shape[0], n_samples)
    im_list = list()
    lab_list = list()
# select images and labels
    for n in range (0, 4): 
        for i in x[n][ix]:
            im_list.append(i)
        for c in y[n][ix]:
            lab_list.append(c)
# generate class labels
    y = ones((n_samples*4, 1))
    return [asarray(im_list), asarray(lab_list)], y

## generate points in latent space

In [16]:
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	z_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = z_input.reshape(n_samples, latent_dim)
	return z_input

## Generate the fake images

In [17]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
# generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
# predict outputs
    images = generator.predict(z_input)
# create class labels
    y = zeros((n_samples, 1))
    return images, y

# save as a plot and save the model

In [18]:
def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
# prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
# scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
# plot images
    for i in range(100):
# define subplot
        plt.subplot(10, 10, i+1)
# turn off axis
        plt.axis('off')
    # plot raw pixel data
        plt.imshow(X[i, :, :, 0])#, cmap='gray_r')
# save plot to file
    filename1 = 'E:/UCF_Crimes/models results/generated_plot_%04d.png' % (step+1)
    plt.savefig(filename1)
    plt.close()
# evaluate the classifier model
    X, y = dataset
    
    for n in range(0, 4):
        _, acc = c_model.evaluate(X[n], y[n], verbose=1)
        print('Classifier Accuracy: %.3f%%' % (acc * 100))
# save the generator model
    filename2 = 'E:/UCF_Crimes/models results/g_model_%04d.h5' % (step+1) 
    g_model.save(filename2)
# save the classifier model
    filename3 = 'E:/UCF_Crimes/models results/c_model_%04d_%01d.h5' % (step+1, (acc *100))
    c_model.save(filename3)
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

# Training the models

In [19]:
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=50, n_batch=50):
# select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset)
    print(X_sup.shape, y_sup.shape)
# calculate the number of batches per training epoch
    bat_per_epo = int(30)
# calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
# calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    
# manually enumerate epochs
    for i in range(n_steps):
# update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples(X_sup, y_sup, half_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
        
# update unsupervised discriminator (d)
        [X_real, y_real], _ = generate_real_samples2(images, labels, half_batch)
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        
# update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        
# summarize loss on this batch
        print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.5f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
# evaluate the model performance every so often
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, c_model, latent_dim, dataset)

## size of the latent space

In [20]:
latent_dim = 75

##  create the discriminator models

In [21]:
d_model, c_model = define_discriminator()

## create the generator

In [22]:
g_model = define_generator(latent_dim)

## Create the GAN

In [23]:
gan_model = define_gan(g_model, d_model)

## Assigning the dataset

In [24]:
dataset = images, labels

## train model

In [25]:
train(g_model, d_model, c_model, gan_model, dataset, latent_dim)

(4760, 120, 120, 3) (4760,)
n_epochs=50, n_batch=50, 1/2=25, b/e=30, steps=1500
>1, c[2.937,24], d[77.829,0.998], g[0.00460]
>2, c[10.731,12], d[58.226,1.000], g[0.00453]
>3, c[7.925,0], d[63.840,1.000], g[0.00000]
>4, c[14.790,8], d[64.349,1.000], g[0.00000]
>5, c[8.959,12], d[54.334,1.000], g[nan]
>6, c[9.247,8], d[55.094,nan], g[nan]
>7, c[nan,16], d[nan,nan], g[nan]
>8, c[nan,4], d[nan,nan], g[nan]
>9, c[nan,4], d[nan,nan], g[nan]
>10, c[nan,16], d[nan,nan], g[nan]
>11, c[nan,4], d[nan,nan], g[nan]
>12, c[nan,4], d[nan,nan], g[nan]
>13, c[nan,8], d[nan,nan], g[nan]
>14, c[nan,0], d[nan,nan], g[nan]
>15, c[nan,8], d[nan,nan], g[nan]
>16, c[nan,12], d[nan,nan], g[nan]
>17, c[nan,8], d[nan,nan], g[nan]
>18, c[nan,8], d[nan,nan], g[nan]
>19, c[nan,0], d[nan,nan], g[nan]
>20, c[nan,16], d[nan,nan], g[nan]
>21, c[nan,16], d[nan,nan], g[nan]
>22, c[nan,12], d[nan,nan], g[nan]
>23, c[nan,12], d[nan,nan], g[nan]
>24, c[nan,12], d[nan,nan], g[nan]
>25, c[nan,8], d[nan,nan], g[nan]
>26, c[nan

C:\Users\MSI\anaconda3\envs\tf-latest\lib\site-packages\matplotlib\image.py:443: UserWarning: Warning: converting a masked element to nan.
  dv = np.float64(self.norm.vmax) - np.float64(self.norm.vmin)
C:\Users\MSI\anaconda3\envs\tf-latest\lib\site-packages\matplotlib\image.py:444: UserWarning: Warning: converting a masked element to nan.
  vmid = np.float64(self.norm.vmin) + dv / 2


313/313 [==============================] - 7s 20ms/step - loss: nan - accuracy: 0.0000e+00: 1s - loss: nan - accuracy: 0.00 - ETA: 1s - loss: nan 
Classifier Accuracy: 0.000%
313/313 [==============================] - 6s 20ms/step - loss: nan - accuracy: 0.0000e+00: 2s
Classifier Accuracy: 0.000%
313/313 [==============================] - 6s 20ms/step - loss: nan - accuracy: 0.0000e+00
Classifier Accuracy: 0.000%
313/313 [==============================] - 6s 20ms/step - loss: nan - accuracy: 0.0000e+00
Classifier Accuracy: 0.000%
>Saved: E:/UCF_Crimes/models results/generated_plot_0030.png, E:/UCF_Crimes/models results/g_model_0030.h5, and E:/UCF_Crimes/models results/c_model_0030_0.h5
>31, c[nan,0], d[nan,nan], g[nan]
>32, c[nan,8], d[nan,nan], g[nan]
>33, c[nan,8], d[nan,nan], g[nan]
>34, c[nan,8], d[nan,nan], g[nan]
>35, c[nan,0], d[nan,nan], g[nan]
>36, c[nan,12], d[nan,nan], g[nan]
>37, c[nan,4], d[nan,nan], g[nan]
>38, c[nan,4], d[nan,nan], g[nan]
>39, c[nan,12], d[nan,nan], g[na

KeyboardInterrupt: 